In [ ]:
# Import packages
import geopandas as gpd
import rasterio as rio
from rasterio.plot import show
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rioxarray as rxr
import xarray as xr
import os

from shapely.geometry import box
from rasterio.features import rasterize
from rasterio import features

# navigate to directory
os.getcwd()

In [ ]:
# Training data path
train_path = ''
# Imagery path
planet_path = ''

In [ ]:
# Read in training data
shapefile = gpd.read_file(train_path)
print(shapefile.crs)

shapefile = shapefile.to_crs('EPSG:32617')
print(shapefile.crs)

In [ ]:
# Read in raster
with rio.open(planet_path) as src:
    stack = src.read()
    print(stack.shape, stack.dtype)
    print(src.crs)

In [ ]:
# rasterize the training data

# Rasterize vector using the shape and coordinate system of the raster
rasterized = features.rasterize([(geom, value) for geom, value in zip(shapefile.geometry, shapefile['class'])],
                                out_shape = src.shape,
                                fill = -99,
                                out = None,
                                transform = src.transform,
                                all_touched = False,
                                #default_value = 1,
                                #nodata = 99,
                                dtype = np.int8) # options np.uint8 or 'int8'

# Plot training data raster
fig, ax = plt.subplots(1, figsize = (10, 10))
show(rasterized, ax = ax)
plt.gca().invert_yaxis()

unique_elements, counts = np.unique(rasterized, return_counts=True)
print(unique_elements) # Output: [1 2 3 4]
print(counts) # Output: [1 2 1 3]

In [ ]:
# turn the input data into an array
# > 7 min
img = np.zeros((src.height, src.width, src.count), dtype=src.dtypes[0])  # Initialize array for all bands

# use np.fill
img = np.full((src.height, src.width, src.count), fill_value=-99, dtype=src.dtypes[0])

# extract the raster values within the polygon 
with rio.open(planet_path) as src:
    for b in range(src.count):
        img[:, :, b] = src.read(b + 1)

# Open the ROI dataset
roi=rasterized.astype(np.int8)

In [ ]:
# Find how many non-zero entries we have -- i.e. how many training data samples?
n_samples = (roi >= 0).sum()
print('We have {n} samples'.format(n=n_samples))

# What are our classification labels?
labels = np.unique(roi[roi >= 0])
print('The training data include {n} classes: {classes}'.format(n=labels.size, 
                                                                classes=labels))
# We will need a "X" matrix containing our features, and a "y" array containing our labels
#     These will have n_samples rows
#     In other languages we would need to allocate these and them loop to fill them, but NumPy can be faster


X = img[roi >= 0, :]  # Select all values of img where roi is greater than 0
y = roi[roi >= 0 ] # Select all values of roi where roi is greater than 0

print('Our X matrix is sized: {sz}'.format(sz=X.shape))
print('Our y array is sized: {sz}'.format(sz=y.shape))

# Mask out bad pixels in planet data
# clear = X[:, 4] <= 1 # Return a boolean with true where values in band 5 are less than 1 

# X = X[clear, :4]  # get rid of band 5 (0:3), keep the first four columns
# y = y[clear]

# X = np.nan_to_num(X, nan=-99) # UNCOMMENT FOR GEE INPUTS

# print('After masking, our X matrix is sized: {sz}'.format(sz=X.shape))
# print('After masking, our y array is sized: {sz}'.format(sz=y.shape))

In [ ]:
unique_elements, counts = np.unique(y, return_counts=True)
print(unique_elements) 
print(counts) 

In [ ]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
# test_size = 0.25 >> i.e. 25% of the data; represents the proportion of the dataset to include in the test split
train_features, test_features, train_labels, test_labels = train_test_split(X, y, 
                                                                            test_size = 0.2, 
                                                                            random_state = 12) 
                                                                            #stratify=y) 


# print things and check stuff out
unique_elements, counts = np.unique(train_labels, return_counts=True)
print(unique_elements) # Output: [1 2 3 4]
print(counts) # Output: [1 2 1 3]

print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize our model with 500 trees
rf = RandomForestClassifier(n_estimators=100, oob_score=True)

rf = rf.fit(train_features, train_labels)

In [ ]:
# preview accuracy
# Confusion Matrix
from sklearn.metrics import confusion_matrix

# Predicting the Test set results
y_train = rf.predict(train_features) 
print(confusion_matrix(train_labels, y_train))

In [ ]:
# preview accuracy
# Confusion Matrix
from sklearn.metrics import confusion_matrix

# Predicting the Test set results
y_pred = rf.predict(test_features) 
print(confusion_matrix(test_labels, y_pred))

# Classificaton Report 
from sklearn.metrics import classification_report
print(classification_report(test_labels, y_pred, digits=4))

In [ ]:
# Take our full image and reshape into long 2d array (nrow * ncol, nband) for classification
new_shape = (img.shape[0] * img.shape[1], img.shape[2])
img_as_array = img[:, :, :np.int8(img.shape[2])].reshape(new_shape)

print('Reshaped from {o} to {n}'.format(o=img.shape, n=img_as_array.shape))

img_as_array = np.nan_to_num(img_as_array)

In [ ]:
# Now predict for each pixel
# >10min
class_prediction = rf.predict(img_as_array)

In [ ]:
class_prediction = class_prediction.reshape(img[:, :, 0].shape)
print('Reshaped back to {}'.format(class_prediction.shape))

In [ ]:
plt.subplot(111)
plt.imshow(class_prediction, cmap=plt.cm.Spectral)
plt.title('Classification result')

In [ ]:
# define function to stretch the color for better viz
def color_stretch(image, index):
    colors = image[:, :, index].astype(np.float64)
    for b in range(colors.shape[2]):
        colors[:, :, b] = rio.plot.adjust_band(colors[:, :, b])
    return colors

In [ ]:
from rasterio.plot import reshape_as_image
reshaped_ps_stack = reshape_as_image(stack)
print(reshaped_ps_stack.shape, type(reshaped_ps_stack))

from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches

# Create colormap
colors = [
    [0.5, 0, 0.5],    # purple (class 0 = nonwater)
    [1, 1, 0]         # yellow (class 1 = water)
]

class_names = ['Non-water', 'Water']

cmap = ListedColormap(colors)
patches = [mpatches.Patch(color=colors[i], label=class_names[i]) for i in range(len(class_names))]

fig, axs = plt.subplots(1, 2, figsize=(20, 20))

img = reshaped_ps_stack.astype('float16')
img_stretched = color_stretch(img, [3,2,1])
axs[0].imshow(img_stretched)
axs[0].set_title("False Color Image")

axs[1].imshow(class_prediction, cmap=cmap)
axs[1].set_title("Class Prediction")

axs[1].legend(handles=patches, loc='lower right', fontsize=12)

In [ ]:
ref = rxr.open_rasterio(planet_path)
class_da = xr.DataArray(class_prediction, dims=("y", "x"))

# Copy metadata from reference
class_da = class_da.rio.write_crs(ref.rio.crs, inplace=True)
class_da = class_da.rio.write_transform(ref.rio.transform(), inplace=True)
class_da = class_da.rio.set_spatial_dims(x_dim=ref.rio.x_dim, y_dim=ref.rio.y_dim, inplace=True)

class_da.astype('float32').rio.to_raster('', driver='GTiff')